In [1]:
%cd ../../

/home/PJWSTK/s14028/engineer/Practical


In [2]:
import keras
import scipy.io as mat

from common import *
from augmentation import add_pmap
from augmentation import augmentation_data
from augmentation import augment_data

from CNN.CNN_v7 import CNN_v7

Using TensorFlow backend.


In [3]:
perspective = mat.loadmat("mall_dataset/perspective_roi.mat")["pMapN"]

perspective /= np.min(perspective)
perspective = np.round(perspective).astype(np.uint8)

train, test = data_sets()
image_tensors = train[0], test[0]
person_coo_tensors = train[1], test[1]
count_matrix = train[2], test[2]

image_train, image_test = image_tensors
person_coo_train, person_coo_test = person_coo_tensors
count_train, count_test = count_matrix
count_train = count_train.astype(np.uint16)
count_test = count_test.astype(np.uint16)

image_train = add_pmap(image_train, perspective)
image_test = add_pmap(image_test, perspective)

In [4]:
cnn_v7 = CNN_v7((480, 640, 4), split_into_parts=20)
cnn_v7.def_model()

In [5]:
images = cnn_v7._prepare_images(image_train)
anwsers = cnn_v7._prepare_anwsers(person_coo_train)

In [6]:
ones_count = (np.sum(anwsers == 1) * 0.25).astype(np.uint32)
zeros_count = (ones_count / 0.25 * 0.75).astype(np.uint32)
validation_length = (zeros_count + ones_count).astype(np.int32)

val_indices = np.concatenate([np.where(anwsers == 0)[0][:zeros_count],
                              np.where(anwsers == 1)[0][:ones_count]])

In [7]:
anwsers[val_indices[zeros_count:]] = 0

In [8]:
val_indices = -(images.shape[0] - val_indices)

In [9]:
generator = keras.preprocessing.image.ImageDataGenerator(rotation_range=20)

augmentation = augmentation_data(image_train, anwsers, 20)
augmented_data = augment_data(generator, augmentation, images, anwsers)
images, anwsers = augmented_data

In [10]:
anwsers[val_indices[zeros_count:]] = 1

images[-validation_length:], images[val_indices] = images[val_indices], images[-validation_length:]
anwsers[-validation_length:], anwsers[val_indices] = anwsers[val_indices], anwsers[-validation_length:]

In [11]:
parallel_model = keras.utils.multi_gpu_model(
        cnn_v7.model,
        gpus=2,
        cpu_merge=False)
parallel_model.compile(
        loss="binary_crossentropy", 
        optimizer="adam", 
        metrics=["accuracy"])

In [12]:
path = "CNN/CNN_v7/weights/cnn_v7_1e_6_{}"

class ModelSaver(keras.callbacks.Callback):
    __path: str
    __current_epoch: int
        
    def __init__(self, path: str, epoch: int = 0):
        self.__path = path
        self.__current_epoch = epoch
        
    def on_epoch_end(self, *args, **kwargs):
        epoch_path = self.__path.format(self.__current_epoch)
        self.__current_epoch += 1
        
        parallel_model.save_weights(epoch_path)

model_saver = ModelSaver(path, 25)

In [13]:
parallel_model.optimizer.lr.assign(1e-4)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-04>

In [14]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 420s 337us/step - loss: 14.3135 - accuracy: 0.9139 - val_loss: 4.2006 - val_accuracy: 0.8791
Epoch 2/5
1244460/1244460 [==============================] - 411s 330us/step - loss: 2.4919 - accuracy: 0.9526 - val_loss: 1.3348 - val_accuracy: 0.8802
Epoch 3/5
1244460/1244460 [==============================] - 410s 329us/step - loss: 0.8052 - accuracy: 0.9531 - val_loss: 0.5907 - val_accuracy: 0.8872
Epoch 4/5
1244460/1244460 [==============================] - 409s 328us/step - loss: 0.3546 - accuracy: 0.9519 - val_loss: 0.3718 - val_accuracy: 0.8985
Epoch 5/5
1244460/1244460 [==============================] - 408s 328us/step - loss: 0.2264 - accuracy: 0.9570 - val_loss: 0.3226 - val_accuracy: 0.8981


In [15]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 413s 332us/step - loss: 0.1787 - accuracy: 0.9621 - val_loss: 0.2814 - val_accuracy: 0.9088
Epoch 2/5
1244460/1244460 [==============================] - 410s 329us/step - loss: 0.1521 - accuracy: 0.9657 - val_loss: 0.2565 - val_accuracy: 0.9129
Epoch 3/5
1244460/1244460 [==============================] - 408s 327us/step - loss: 0.1345 - accuracy: 0.9683 - val_loss: 0.2370 - val_accuracy: 0.9177
Epoch 4/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.1221 - accuracy: 0.9703 - val_loss: 0.2297 - val_accuracy: 0.9198
Epoch 5/5
1244460/1244460 [==============================] - 408s 328us/step - loss: 0.1119 - accuracy: 0.9721 - val_loss: 0.2180 - val_accuracy: 0.9241


In [16]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.1038 - accuracy: 0.9737 - val_loss: 0.1954 - val_accuracy: 0.9334
Epoch 2/5
1244460/1244460 [==============================] - 402s 323us/step - loss: 0.0976 - accuracy: 0.9745 - val_loss: 0.1962 - val_accuracy: 0.9324
Epoch 3/5
1244460/1244460 [==============================] - 403s 324us/step - loss: 0.0915 - accuracy: 0.9759 - val_loss: 0.1991 - val_accuracy: 0.9283
Epoch 4/5
1244460/1244460 [==============================] - 403s 324us/step - loss: 0.0867 - accuracy: 0.9767 - val_loss: 0.1748 - val_accuracy: 0.9398
Epoch 5/5
1244460/1244460 [==============================] - 402s 323us/step - loss: 0.0825 - accuracy: 0.9774 - val_loss: 0.1606 - val_accuracy: 0.9450


In [17]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0788 - accuracy: 0.9783 - val_loss: 0.1485 - val_accuracy: 0.9502
Epoch 2/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0758 - accuracy: 0.9788 - val_loss: 0.1644 - val_accuracy: 0.9399
Epoch 3/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0723 - accuracy: 0.9796 - val_loss: 0.1402 - val_accuracy: 0.9532
Epoch 4/5
1244460/1244460 [==============================] - 408s 327us/step - loss: 0.0702 - accuracy: 0.9801 - val_loss: 0.1431 - val_accuracy: 0.9483
Epoch 5/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0676 - accuracy: 0.9806 - val_loss: 0.1501 - val_accuracy: 0.9440


In [18]:
parallel_model.optimizer.lr.assign(1e-5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-05>

In [17]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0788 - accuracy: 0.9783 - val_loss: 0.1485 - val_accuracy: 0.9502
Epoch 2/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0758 - accuracy: 0.9788 - val_loss: 0.1644 - val_accuracy: 0.9399
Epoch 3/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0723 - accuracy: 0.9796 - val_loss: 0.1402 - val_accuracy: 0.9532
Epoch 4/5
1244460/1244460 [==============================] - 408s 327us/step - loss: 0.0702 - accuracy: 0.9801 - val_loss: 0.1431 - val_accuracy: 0.9483
Epoch 5/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0676 - accuracy: 0.9806 - val_loss: 0.1501 - val_accuracy: 0.9440


In [13]:
last_model_epoch = 24
last_model_path = path.format(last_model_epoch)

parallel_model.load_weights(last_model_path)

In [14]:
parallel_model.optimizer.lr.assign(1e-5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-05>

In [15]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 415s 333us/step - loss: 0.0615 - accuracy: 0.9829 - val_loss: 0.1286 - val_accuracy: 0.9544
Epoch 2/5
1244460/1244460 [==============================] - 408s 328us/step - loss: 0.0596 - accuracy: 0.9836 - val_loss: 0.1200 - val_accuracy: 0.9598
Epoch 3/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0583 - accuracy: 0.9839 - val_loss: 0.1167 - val_accuracy: 0.9616
Epoch 4/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0577 - accuracy: 0.9841 - val_loss: 0.1130 - val_accuracy: 0.9633
Epoch 5/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0567 - accuracy: 0.9845 - val_loss: 0.1117 - val_accuracy: 0.9631


In [16]:
parallel_model.optimizer.lr.assign(1e-6)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=1e-06>

In [17]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0555 - accuracy: 0.9848 - val_loss: 0.1128 - val_accuracy: 0.9624
Epoch 2/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0555 - accuracy: 0.9848 - val_loss: 0.1115 - val_accuracy: 0.9629
Epoch 3/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0553 - accuracy: 0.9850 - val_loss: 0.1105 - val_accuracy: 0.9640
Epoch 4/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0556 - accuracy: 0.9848 - val_loss: 0.1109 - val_accuracy: 0.9635
Epoch 5/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0553 - accuracy: 0.9848 - val_loss: 0.1105 - val_accuracy: 0.9637


In [18]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0551 - accuracy: 0.9851 - val_loss: 0.1097 - val_accuracy: 0.9641
Epoch 2/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0551 - accuracy: 0.9850 - val_loss: 0.1106 - val_accuracy: 0.9636
Epoch 3/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0551 - accuracy: 0.9850 - val_loss: 0.1107 - val_accuracy: 0.9634
Epoch 4/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0550 - accuracy: 0.9851 - val_loss: 0.1105 - val_accuracy: 0.9636
Epoch 5/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0547 - accuracy: 0.9851 - val_loss: 0.1096 - val_accuracy: 0.9639


In [19]:
parallel_model.fit(images[:-validation_length], anwsers[:-validation_length], batch_size=3000, epochs=5,
                   validation_data=(images[-validation_length:], anwsers[-validation_length:]),
                   callbacks=[model_saver])

Train on 1244460 samples, validate on 46216 samples
Epoch 1/5
1244460/1244460 [==============================] - 407s 327us/step - loss: 0.0547 - accuracy: 0.9852 - val_loss: 0.1093 - val_accuracy: 0.9642
Epoch 2/5
1244460/1244460 [==============================] - 402s 323us/step - loss: 0.0548 - accuracy: 0.9851 - val_loss: 0.1091 - val_accuracy: 0.9642
Epoch 3/5
1244460/1244460 [==============================] - 402s 323us/step - loss: 0.0545 - accuracy: 0.9852 - val_loss: 0.1085 - val_accuracy: 0.9644
Epoch 4/5
1244460/1244460 [==============================] - 402s 323us/step - loss: 0.0545 - accuracy: 0.9852 - val_loss: 0.1091 - val_accuracy: 0.9642
Epoch 5/5
1244460/1244460 [==============================] - 403s 324us/step - loss: 0.0545 - accuracy: 0.9853 - val_loss: 0.1082 - val_accuracy: 0.9648
